In [1]:
pip install pandas sqlalchemy pymysql pyodbc openpyxl

In [8]:
# Importation des différentes librairies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import urllib

In [9]:
# === Fonction de connexion MySQL ===
def connect_mysql(host, user, password, db):
    try:
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")
        engine.connect()
        print("✅ Connexion MySQL réussie")
        return engine
    except SQLAlchemyError as e:
        print(f"❌ Erreur connexion MySQL : {e}")
        return None

In [10]:
# === Fonction de connexion SQL Server ===
def connect_sqlserver(server, database, user, password):
    try:
        params = urllib.parse.quote_plus(
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={server};DATABASE={database};UID={user};PWD={password}"
        )
        engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
        engine.connect()
        print("✅ Connexion SQL Server réussie")
        return engine
    except SQLAlchemyError as e:
        print(f"❌ Erreur connexion SQL Server : {e}")
        return None

In [11]:
# === Fonction de transfert de table ===
def transfer_table(table_name, source_engine, dest_engine):
    try:
        print(f"\n🔄 Transfert de {table_name} en cours...")
        df = pd.read_sql(f"SELECT * FROM {table_name}", con=source_engine)
        df.to_sql(table_name, con=dest_engine, if_exists="append", index=False)
        print(f"✅ Table {table_name} transférée avec succès.")
    except SQLAlchemyError as e:
        print(f"❌ Erreur pendant le transfert de {table_name} : {e}")

In [12]:
# === Paramètres à modifier ===
MYSQL_HOST = "localhost"
MYSQL_USER = "root"
MYSQL_PWD = ""
MYSQL_DB = "dataprojet"

SQLSERVER_HOST = "OUMAR-CISS\SQLEXPRESS"
SQLSERVER_DB = "base_projet"
SQLSERVER_USER = "test"
SQLSERVER_PWD = "1234"

In [14]:
# === Connexions ===
mysql_engine = connect_mysql(MYSQL_HOST, MYSQL_USER, MYSQL_PWD, MYSQL_DB)
sqlserver_engine = connect_sqlserver(SQLSERVER_HOST, SQLSERVER_DB, SQLSERVER_USER, SQLSERVER_PWD)

✅ Connexion MySQL réussie
✅ Connexion SQL Server réussie


In [19]:
# === Transfert des tables MySQL => SQL Server ===
if mysql_engine and sqlserver_engine:
    tables = ["segment", "canal", "branche", "produit", "client", "intermediaire", "production"]
    for table in tables:
        transfer_table(table, mysql_engine, sqlserver_engine)
else:
    print("⛔ Connexion à l’une des bases échouée. Opération annulée.")


✅ Connexion MySQL réussie
✅ Connexion SQL Server réussie

🔄 Transfert de segment en cours...
❌ Erreur pendant le transfert de segment : (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__segment__0CFFA014759CEBF8'. Cannot insert duplicate key in object 'dbo.segment'. The duplicate key value is (1). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO segment (id_segment, libelle) VALUES (?, ?)]
[parameters: ((1, 'PARTICULIER'), (2, 'ENTREPRISES'), (3, 'AUTRES'))]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

🔄 Transfert de canal en cours...
❌ Erreur pendant le transfert de canal : (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__canal__D4C01CC8F84B95C4'. Cannot insert duplicate key in object 'dbo.canal'

In [22]:
import pandas as pd
from sqlalchemy.exc import SQLAlchemyError

def load_objectif_excel(excel_path, sqlserver_engine):
    try:
        print("\n📥 Chargement du fichier Excel Objectif.xlsx")
        
        # Chargement du fichier Excel
        df = pd.read_excel(excel_path)

        # Renommage explicite des colonnes pour correspondre à la table SQL Server
        df = df.rename(columns={
            "Année": "annee",
            "Exercice": "exercice",
            "Id_segment": "id_segment",
            "Segment": "segment",
            "Id_branche": "id_branche",
            "Branche": "branche",
            "id_produit": "id_produit",
            "Produit": "produit",
            "Id_intermediaire": "id_intermediaire",
            "Intermediaire": "intermediaire",
            "Id_Canal": "id_canal",
            "canal": "canal",
            "Objectif_CA": "objectif_ca"
        })

        # Chargement dans SQL Server
        df.to_sql("objectif", con=sqlserver_engine, if_exists="append", index=False)
        print("✅ Objectif chargé avec succès dans SQL Server.")

    except SQLAlchemyError as e:
        print(f"❌ Erreur SQLAlchemy : {e}")
    except Exception as e:
        print(f"❌ Erreur générale : {e}")


In [23]:
load_objectif_excel("Objectif.xlsx", sqlserver_engine)


📥 Chargement du fichier Excel Objectif.xlsx
✅ Objectif chargé avec succès dans SQL Server.


In [16]:
# Création de la variable chiffre d'affaires (CA)

# Lire la table production depuis MySQL
df_prod = pd.read_sql("SELECT * FROM production", mysql_engine)

# Calcul de la colonne CA (exemple : prime_nette + taxe)
df_prod["CA"] = df_prod["prime_nette"] + df_prod["cout_police"]

# Charger dans SQL Server (remplacer ou compléter selon ton besoin)
df_prod.to_sql("production", sqlserver_engine, if_exists="replace", index=False)  # ou append

print("✅ Table production mise à jour avec colonne CA.")


✅ Table production mise à jour avec colonne CA.


In [24]:
# Chargement de la table production
df = pd.read_sql("SELECT * FROM production", sqlserver_engine)  # ou mysql_engine

# Conversion des dates
df["date_effet"] = pd.to_datetime(df["date_effet"])
df["date_fin"] = pd.to_datetime(df["date_fin"])

# Trie des contrats
df = df.sort_values(["client_id_client", "produit_id_produit", "date_effet"])

# Ajout de la date du contrat suivant
df["date_effet_suivante"] = df.groupby(["client_id_client", "produit_id_produit"])["date_effet"].shift(-1)

# Création de la variable de statut de renouvellement
df["etat_renouvellement"] = df.apply(
    lambda row: "non renouvelé" if pd.isna(row["date_effet_suivante"]) 
                                     or row["date_fin"] < row["date_effet_suivante"] - pd.Timedelta(days=1)
                else "renouvelé",
    axis=1
)

# 6. Affichage de vérification
print(df[["client_id_client", "produit_id_produit", "date_effet", "date_fin", "etat_renouvellement"]])


       client_id_client  produit_id_produit date_effet   date_fin  \
62185                 1                  12 2021-11-01 2022-10-31   
62186                 1                  12 2023-01-18 2024-01-17   
83194                 2                  12 2023-09-26 2024-03-25   
74431                 3                  13 2022-10-01 2023-09-30   
40428                 4                  12 2020-09-18 2021-09-17   
...                 ...                 ...        ...        ...   
26677             26730                  13 2021-03-18 2021-09-17   
36517             26730                  13 2021-03-21 2021-09-20   
52394             26730                  13 2021-06-05 2021-12-04   
61372             26731                  12 2021-12-03 2022-12-02   
61373             26731                  12 2022-12-03 2023-12-02   

      etat_renouvellement  
62185       non renouvelé  
62186       non renouvelé  
83194       non renouvelé  
74431       non renouvelé  
40428       non renouvelé  
...

In [25]:
df.to_sql("production", con=sqlserver_engine, if_exists="replace", index=False)


-1